In [3]:
import sys
import os
import json
import chess
from time import sleep, time
from datetime import datetime, timedelta
from random import random, choice
from IPython.display import clear_output

# Bots
import random_bot
#import final_bot
import basic_bot_1_11

dev = True

### Functions

def print_time(to_print, 
               t=0, 
               clear=True):
    if clear: clear_output()
    print to_print, '\n'
    sys.stdout.flush()
    sleep(t)
    pass

def get_config(board, me):
    cmap = {
        ( 'WIN', 'WHITE'):  'W_win',
        ('DRAW', 'WHITE'): 'W_draw',
        ('LOSE', 'WHITE'): 'W_lose',
        ( 'WIN', 'BLACK'):  'B_win',
        ('DRAW', 'BLACK'): 'B_draw',
        ('LOSE', 'BLACK'): 'B_lose'}

    result = None
    if board.is_checkmate():
        if not board.turn == me:
            result = 'WIN'
        else:
            result = 'LOSE'
    else:
        result = 'DRAW'

    return cmap[(result,turn_color[me])]

def get_status(board):
    result = ""
    if board.is_game_over():
        if board.is_checkmate(): result = 'Checkmate! ' + turn_color[not board.turn] + ' wins'
        if board.is_stalemate(): result = 'Stalemate! DRAW'
        if board.is_insufficient_material(): result = 'Insufficient material. DRAW', 
        if board.is_seventyfive_moves(): result = 'Seventyfive moves without capture or pawn move. DRAW'
        if board.is_fivefold_repetition(): result = 'Fivefold repetition. DRAW'
    else:
        result = turn_color[board.turn], 'to move'
    return result

### Parameters

turn_color = {
    True:'WHITE',
    False:'BLACK',
    None: None
}

def new_state():
    state = {
        'played_since': '',
        'game_desc':    '',
        'fen':          '',
        'turn_desc':    '',
        'W_win':         0,
        'W_draw':        0,
        'W_lose':        0,
        'B_win':         0,
        'B_draw':        0,
        'B_lose' :       0
    }
    return state

def init_state():
    state = None
    if os.path.exists('../html/state'):
        with open('../html/state', 'r') as f:
            state = json.loads(f.read())
    if not state:
        state = new_state()
    return state

def write_state(state):
    with open('../html/state', 'w') as f:
        f.write(json.dumps(state, indent=4))
    pass

state = init_state()

# The Arena
me_bot = basic_bot_1_11
he_bot = random_bot
players = [chess.WHITE, chess.BLACK]
min_turn_time = 0.1

# In[ ]:
if not state['played_since']:
    state['played_since'] = (datetime.fromtimestamp(time())+timedelta(hours=1)).strftime('%Y-%m-%d %H:%M:%S')


In [7]:
r = 0.05

value_piece = {
    chess.PAWN:   1.0,
    chess.KNIGHT: 2.95,
    chess.BISHOP: 3.05,
    chess.ROOK:   5.0,
    chess.QUEEN:  9.0,
    chess.KING:   0.0
}

# Position weight
wp = [[1,1,1,1,1,1,1,1],
      [1,2,2,2,2,2,2,1],
      [1,2,3,3,3,3,2,1],
      [1,2,3,5,5,3,2,1],
      [1,2,3,5,5,3,2,1],
      [1,2,3,3,3,3,2,1],
      [1,2,2,2,2,2,2,1],
      [1,1,1,1,1,1,1,1]]
WP = sum(map(sum, wp))

# Attack weight (for black)
wa0 = [[3,4,4,4,4,4,4,3],
       [3,4,4,4,4,4,4,3],
       [2,3,3,3,3,3,3,2],
       [2,3,3,3,3,3,3,2],
       [1,2,2,2,2,2,2,1],
       [1,2,2,2,2,2,2,1],
       [1,1,1,1,1,1,1,1],
       [1,1,1,1,1,1,1,1]]
WA = sum(map(sum, wa0))
wa = {}
wa[chess.WHITE] = wa0[::-1]
wa[chess.BLACK] = wa0


square_weight = {'pos': {},
                 'atk': {chess.WHITE: {},
                         chess.BLACK: {}}
                 }
for i in xrange(8):
    for j in xrange(8):
        square_weight['pos'][chess.square(j,i)] = wp[i][j]/float(WP)
        square_weight['atk'][chess.WHITE][chess.square(j,i)] = wa[chess.WHITE][i][j]/float(WA)
        square_weight['atk'][chess.BLACK][chess.square(j,i)] = wa[chess.BLACK][i][j]/float(WA)

        
def eval_player(board, player):
    
    score_material = 0
    score_presence = 0
    score_attacks = 0
    
    piece_squares = chess.SquareSet()
    attack_squares = chess.SquareSet()
    for piece_type in xrange(1,7):
        piece_type_squares = board.pieces(piece_type, player)
        piece_squares = piece_squares.union(piece_type_squares)
        for square in piece_type_squares:
            
            score_material += value_piece[piece_type]
            score_presence += square_weight['pos'][square]
            attack_squares = attack_squares.union(board.attacks(square))
    
    attack_squares = attack_squares.difference(piece_squares)

    score_attacks = reduce(lambda x,y: x+square_weight['atk'][player][y], attack_squares, 0)

    return (score_material, (1-r)*score_presence, r*score_attacks)

In [11]:
while True:
    
    board = chess.Board() # Set up the board
    me    = choice(players)  # My color
    turn  = 0

    cpu_time = {chess.WHITE: 0, chess.BLACK: 0}

    state['game_desc'] = 'Bot plays ' + turn_color[me] + ', ' + turn_color[not me] + ' is random\n'
        
    while not board.is_game_over():
        
        t0 = time()
        turn+=1

        state['fen'] = board.fen()
        
        tt = time()
        if board.turn == me:
            state['turn_desc'] = str(1+turn/2) + ' - My move, ' + turn_color[board.turn]
            move = me_bot.move(board) # My move!
        else:
            state['turn_desc'] = str(1+turn/2) + ' - His move, ' + turn_color[board.turn]
            move = he_bot.move(board) # My opponent's move
        ct = time() - tt
        cpu_time[board.turn] += ct

        board.push(move) # Make the move
        
        if dev: 
            print_time(board)
            print "(%.2f,%.5f,%.5f)" % eval_player(board, chess.WHITE)
            print "(%.2f,%.5f,%.5f)" % eval_player(board, chess.BLACK)
            print state
            sys.stdout.flush()

        t1 = time()
        if t1-t0 < min_turn_time:
            sleep(min_turn_time - (t1-t0))

        write_state(state)
    
    state['fen'] = board.fen()
    state['game_desc'] = get_status(board)

    result = None
    if board.is_checkmate(): 
        result = turn_color[not board.turn]

    state[get_config(board, me)] += 1

    write_state(state)

    #log_entry = {
    #    'move_stack': ','.join([str(move) for move in board.move_stack]),
    #    'winner': turn_color[not board.turn] if board.is_checkmate() else None,
    #    'WHITE': {
    #        'bot': me_bot.name() if me == chess.WHITE else he_bot.name(),
    #        'cpu_time': cpu_time[chess.WHITE]
    #        },
    #    'BLACK': {
    #        'bot': me_bot.name() if me == chess.BLACK else he_bot.name(),
    #        'cpu_time': cpu_time[chess.BLACK]
    #        },
    #    }
    #
    #with open('log_entry') as f:
    #    f.write(json.dumps(log_entry, separators=(',', ': ')))
    #    write_command = 's3cmd put log_entry %i s3://larsbk/%s/%s/%s/%i' % (i,
    #                                                                        log_entry[winner],
    #                                                                        log_entry[not loser],
    #                                                                        color,i)
    #
    #
    #write_command = 's3cmd put'
    #    
    sleep(8)



. n b q k b n r
. p p . p p . p
. . . p . . p .
p . . . . . . .
. . B P P . . .
. . . . . . . .
P P P . . P P P
R N B Q K . N R 

(39.00,0.22984,0.01757)
(34.00,0.16855,0.00912)
{u'B_draw': 2, u'fen': '1nbqkbnr/1ppppp1p/6p1/p7/2BPP3/8/PPP2PPP/RNBQK1NR b KQk - 1 4', u'W_win': 4, u'played_since': u'2015-12-19 22:33:51', u'turn_desc': '5 - His move, BLACK', u'B_lose': 0, u'W_lose': 0, u'B_win': 6, u'W_draw': 3, u'game_desc': 'Bot plays WHITE, BLACK is random\n'}


KeyboardInterrupt: 

In [ ]:
board = chess.Board()

In [3]:
pieces = board.pieces(2, chess.WHITE)

In [11]:
print pieces

. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. 1 . . . . 1 .


In [18]:
pieces

SquareSet(0b1000010)

In [20]:
S = chess.SquareSet()
for s in pieces:
    S = S.union(board.attacks(s))
print S

. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
1 . 1 . . 1 . 1
. . . 1 1 . . .
. . . . . . . .


In [21]:
w = [[1,1,1,1,1,1,1,1],
     [1,2,2,2,2,2,2,1],
     [1,2,3,3,3,3,2,1],
     [1,2,3,5,5,3,2,1],
     [1,2,3,5,5,3,2,1],
     [1,2,3,3,3,3,2,1],
     [1,2,2,2,2,2,2,1],
     [1,1,1,1,1,1,1,1]]

In [29]:
import time

In [37]:
t = time.time()
print sum([sum(s) for s in w])
print time.time()-t
t = time.time()
print reduce(lambda x,y: x+sum(y), w, 0)
print time.time()-t
t = time.time()
print sum(map(sum, w))
print time.time()-t

124
0.000204086303711
124
0.000207901000977
124
0.000157117843628


In [56]:
square_importance = {}
for i,r in enumerate(w):
    for j,c in enumerate(r):
        square_importance[chess.square(i,j)] = c

value_piece = {
    chess.PAWN:   1.0,
    chess.KNIGHT: 2.9,
    chess.BISHOP: 3.1,
    chess.ROOK:   5.0,
    chess.QUEEN:  9.0,
    chess.KING:   0.0
}
score_material = 0
score_presence = 0

piece_squares = chess.SquareSet()
attack_sqares = chess.SquareSet()
for piece_type in xrange(1,7):
    piece_type_squares = board.pieces(piece_type, chess.WHITE)
    piece_squares = piece_squares.union(piece_type_squares)
    for square in piece_type_squares:
        
        score_material += value_piece[piece_type]
        score_presence += square_importance[square]
        attack_squares = attack_squares.union(board.attacks(square))
        
attack_squares = attack_squares.difference(piece_squares)

print attack_squares

. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
1 1 1 1 1 1 1 1
. . . . . . . .
. . . . . . . .


In [54]:
atk = 0
for s in attack_squares:
    atk += square_importance[s]
print atk
    

38


In [51]:
reduce(lambda x,y: x+square_importance[y], attack_squares, 0)

38

In [50]:
attack_squares

SquareSet(0b111111111111111101111110)